## Analyze A/B Test Results

This project will assure you have mastered the subjects covered in the statistics lessons.  The hope is to have this project be as comprehensive of these topics as possible.  Good luck!

## Table of Contents
- [Introduction](#intro)
- [Part I - Probability](#probability)
- [Part II - A/B Test](#ab_test)
- [Part III - Regression](#regression)


<a id='intro'></a>
### Introduction

A/B tests are very commonly performed by data analysts and data scientists.  It is important that you get some practice working with the difficulties of these 

For this project, you will be working to understand the results of an A/B test run by an e-commerce website.  Your goal is to work through this notebook to help the company understand if they should implement the new page, keep the old page, or perhaps run the experiment longer to make their decision.

**As you work through this notebook, follow along in the classroom and answer the corresponding quiz questions associated with each question.** The labels for each classroom concept are provided for each question.  This will assure you are on the right track as you work through the project, and you can feel more confident in your final submission meeting the criteria.  As a final check, assure you meet all the criteria on the [RUBRIC](https://review.udacity.com/#!/projects/37e27304-ad47-4eb0-a1ab-8c12f60e43d0/rubric).

<a id='probability'></a>
#### Part I - Probability

To get started, let's import our libraries.

In [187]:
import pandas as pd
import numpy as np
import random
import matplotlib.pyplot as plt
%matplotlib inline
#We are setting the seed to assure you get the same answers on quizzes as we set up
random.seed(42)

`1.` Now, read in the `ab_data.csv` data. Store it in `df`.  **Use your dataframe to answer the questions in Quiz 1 of the classroom.**

a. Read in the dataset and take a look at the top few rows here:

In [188]:
df = pd.read_csv('ab_data.csv') # reading the data
df.head(10) # showing a sample

,user_id,timestamp,group,landing_page,converted
0,851104,2017-01-21 22:11:48.556739,control,old_page,0
1,804228,2017-01-12 08:01:45.159739,control,old_page,0
2,661590,2017-01-11 16:55:06.154213,treatment,new_page,0
3,853541,2017-01-08 18:28:03.143765,treatment,new_page,0
4,864975,2017-01-21 01:52:26.210827,control,old_page,1
5,936923,2017-01-10 15:20:49.083499,control,old_page,0
6,679687,2017-01-19 03:26:46.940749,treatment,new_page,1
7,719014,2017-01-17 01:48:29.539573,control,old_page,0
8,817355,2017-01-04 17:58:08.979471,treatment,new_page,1
9,839785,2017-01-15 18:11:06.610965,treatment,new_page,1


b. Use the below cell to find the number of rows in the dataset.

In [189]:
len(df)

294478

In [190]:
# another way:
df.shape[0]

294478

c. The number of unique users in the dataset.

In [191]:
df.user_id.nunique() 

290584

d. The proportion of users converted.

In [192]:
len(df[df.converted == 1])/len(df)

0.11965919355605512

e. The number of times the `new_page` and `treatment` don't line up.

In [231]:
df.groupby(['group','landing_page']).size().iloc[[0,3]].sum()

3893

In [233]:
# another way:
len(df[~(((df.group == 'treatment') & (df.landing_page == 'new_page'))|((df.group == 'control') & (df.landing_page == 'old_page')))])

3893

f. Do any of the rows have missing values?

In [195]:
df.isna().sum().sum() # if zero then no

0

`2.` For the rows where **treatment** is not aligned with **new_page** or **control** is not aligned with **old_page**, we cannot be sure if this row truly received the new or old page.  Use **Quiz 2** in the classroom to provide how we should handle these rows.  

a. Now use the answer to the quiz to create a new dataset that meets the specifications from the quiz.  Store your new dataframe in **df2**.

In [196]:
df2 = df[(((df.group == 'treatment') & (df.landing_page == 'new_page'))|((df.group == 'control') & (df.landing_page == 'old_page')))].copy()
df2.head(10)

,user_id,timestamp,group,landing_page,converted
0,851104,2017-01-21 22:11:48.556739,control,old_page,0
1,804228,2017-01-12 08:01:45.159739,control,old_page,0
2,661590,2017-01-11 16:55:06.154213,treatment,new_page,0
3,853541,2017-01-08 18:28:03.143765,treatment,new_page,0
4,864975,2017-01-21 01:52:26.210827,control,old_page,1
5,936923,2017-01-10 15:20:49.083499,control,old_page,0
6,679687,2017-01-19 03:26:46.940749,treatment,new_page,1
7,719014,2017-01-17 01:48:29.539573,control,old_page,0
8,817355,2017-01-04 17:58:08.979471,treatment,new_page,1
9,839785,2017-01-15 18:11:06.610965,treatment,new_page,1


In [197]:
# Double Check all of the correct rows were removed - this should be 0
df2[((df2['group'] == 'treatment') == (df2['landing_page'] == 'new_page')) == False].shape[0]

0

`3.` Use **df2** and the cells below to answer questions for **Quiz3** in the classroom.

a. How many unique **user_id**s are in **df2**?

In [198]:
df2.user_id.nunique()

290584

b. There is one **user_id** repeated in **df2**.  What is it?

In [199]:
repeated_id = df2.user_id.value_counts().head(1).index[0] # using value_counts method to count how many times each id repeated and get the first id that will be the most repeated id.
repeated_id

773192

c. What is the row information for the repeat **user_id**? 

In [200]:
df2[df2.user_id == repeated_id]

,user_id,timestamp,group,landing_page,converted
1899,773192,2017-01-09 05:37:58.781806,treatment,new_page,0
2893,773192,2017-01-14 02:55:59.590927,treatment,new_page,0


d. Remove **one** of the rows with a duplicate **user_id**, but keep your dataframe as **df2**.

In [201]:
df2.drop([1899], axis = 0, inplace= True) # removing row at index 1899

`4.` Use **df2** in the below cells to answer the quiz questions related to **Quiz 4** in the classroom.

a. What is the probability of an individual converting regardless of the page they receive?

In [202]:
(df2.groupby(['landing_page','converted']).size()/len(df2)).reset_index() # grrouping by landing_page and converted so I can see how many groupes repeated there are

,landing_page,converted,0
0,new_page,0,0.440651
1,new_page,1,0.059411
2,old_page,0,0.439752
3,old_page,1,0.060186


In [203]:
import plotly.express as px

def percentages(cols, data = df2,size=(500,500)): # a function plots the precentages of some columns to others.
    grouped = data[cols].groupby(cols).size().sort_values(ascending = False).reset_index().rename(columns={0:'percentage'})
    list_of_groups = []
    for col in cols[:-1]:
        for unique in data[col].unique():
            list_of_groups.append(grouped[grouped[col] == unique])
    grouped_t_c = pd.concat(list_of_groups)
    
    fig = px.sunburst(grouped_t_c, path=cols, values = 'percentage')
    
    fig.update_traces(textinfo='label+percent entry')
    fig.update_layout(autosize=False, width=size[0], height=size[1])
    fig.show()

percentages(['landing_page','converted']) # it's an interactive chart that you can click on and see the probabilities from one to another

b. Given that an individual was in the `control` group, what is the probability they converted?

In [204]:
len(df2[(df2.group == 'control')&(df2.converted == 1)])/len(df2[(df2.group == 'control')])

0.1203863045004612

c. Given that an individual was in the `treatment` group, what is the probability they converted?

In [205]:
len(df2[(df2.group == 'treatment')&(df2.converted == 1)])/len(df2[(df2.group == 'treatment')])

0.11880806551510564

d. What is the probability that an individual received the new page?

In [206]:
len(df2[df2.landing_page == 'new_page'])/len(df2)

0.5000619442226688

e. Consider your results from a. through d. above, and explain below whether you think there is sufficient evidence to say that the new treatment page leads to more conversions.

**Your answer goes here.**
##### Although the old page has approximately 0.002% more converting, still this value is too small to make up a decision according to it.

<a id='ab_test'></a>
### Part II - A/B Test

Notice that because of the time stamp associated with each event, you could technically run a hypothesis test continuously as each observation was observed.  

However, then the hard question is do you stop as soon as one page is considered significantly better than another or does it need to happen consistently for a certain amount of time?  How long do you run to render a decision that neither page is better than another?  

These questions are the difficult parts associated with A/B tests in general.  


`1.` For now, consider you need to make the decision just based on all the data provided.  If you want to assume that the old page is better unless the new page proves to be definitely better at a Type I error rate of 5%, what should your null and alternative hypotheses be?  You can state your hypothesis in terms of words or in terms of **$p_{old}$** and **$p_{new}$**, which are the converted rates for the old and new pages.

**Put your answer here.**


The null hypothesis will be that if the p-value is less than 5%, the old page has a higher chance of converting users, and the alternative hypothesis will be if the p-value is equal to or greater than 5%, then the new page has a higher chance of converting users.

`2.` Assume under the null hypothesis, $p_{new}$ and $p_{old}$ both have "true" success rates equal to the **converted** success rate regardless of page - that is $p_{new}$ and $p_{old}$ are equal. Furthermore, assume they are equal to the **converted** rate in **ab_data.csv** regardless of the page. <br><br>

Use a sample size for each page equal to the ones in **ab_data.csv**.  <br><br>

Perform the sampling distribution for the difference in **converted** between the two pages over 10,000 iterations of calculating an estimate from the null.  <br><br>

Use the cells below to provide the necessary parts of this simulation.  If this doesn't make complete sense right now, don't worry - you are going to work through the problems below to complete this problem.  You can use **Quiz 5** in the classroom to make sure you are on the right track.<br><br>

a. What is the **convert rate** for $p_{new}$ under the null? 

the Null hypothesis assumes that there is not difference between the conversion rates of old and new page. Thus, we will use the whole dataset to calculate the convertion rate for the new and old page.

In [235]:
pn = (df2.converted == 1).mean()
pn

0.11959708724499628

b. What is the **convert rate** for $p_{old}$ under the null? <br><br>

In [236]:
po = pn
po

0.11959708724499628

c. What is $n_{new}$?

In [209]:
nn = len(df2[df2.landing_page == 'new_page'])

d. What is $n_{old}$?

In [210]:
no = len(df2[df2.landing_page == 'old_page'])

e. Simulate $n_{new}$ transactions with a convert rate of $p_{new}$ under the null.  Store these $n_{new}$ 1's and 0's in **new_page_converted**.

In [211]:
new_converted = np.random.choice(2, size=nn ,p=[pn,1 - pn]).mean() # using np.random.choice for sampling
new_converted

0.8804555777303695

f. Simulate $n_{old}$ transactions with a convert rate of $p_{old}$ under the null.  Store these $n_{old}$ 1's and 0's in **old_page_converted**.

In [212]:
oldconverted = np.random.choice(2, size=no ,p=[po,1 - po]).mean()
oldconverted

0.881389649902942

g. Find $p_{new}$ - $p_{old}$ for your simulated values from part (e) and (f).

In [213]:
new_converted - oldconverted

-0.0009340721725724643

h. Simulate 10,000 $p_{new}$ - $p_{old}$ values using this same process similarly to the one you calculated in parts **a. through g.** above.  Store all 10,000 values in a numpy array called **p_diffs**.

In [214]:
p_diffs, new_means, old_means = [], [], []

for _ in range (10000):

    old = np.random.choice(2, size=no ,p=[po,1 - po])
    new = np.random.choice(2, size=nn ,p=[pn,1 - pn])

    old_means.append(old.mean())
    new_means.append(new.mean())

    p_diffs.append(new.mean() - old.mean())

i. Plot a histogram of the **p_diffs**.  Does this plot look like what you expected?  Use the matching problem in the classroom to assure you fully understand what was computed here.

In [215]:
p_diffs = np.array(p_diffs) # converting to array before ploting

px.histogram(p_diffs, nbins=20) # using plotly.express.histogram

Yes it's what I expected as it's a normal distribution.

j. What proportion of the **p_diffs** are greater than the actual difference observed in **ab_data.csv**?

In [216]:
ad = df2[df2.landing_page == 'new_page'].converted.mean() - df2[df2.landing_page == 'old_page'].converted.mean()
(p_diffs > ad).mean() # p value

0.9006

k. In words, explain what you just computed in part **j.**  What is this value called in scientific studies?  What does this value mean in terms of whether or not there is a difference between the new and old pages?

**Put your answer here.**

90.06% is the proportion of the p_diffs that are greater than the actual difference observed in ab_data.csv. this value is also called **p-value** In scientific studies. This value (that is greater than 0.05 or 5%) means that we cannot reject the null hypothesis and that we do not have sufficient evidence that the new_page has a higher conversion rate than the old_page.

l. We could also use a built-in to achieve similar results.  Though using the built-in might be easier to code, the above portions are a walkthrough of the ideas that are critical to correctly thinking about statistical significance. Fill in the below to calculate the number of conversions for each page, as well as the number of individuals who received each page. Let `n_old` and `n_new` refer the the number of rows associated with the old page and new pages, respectively.

In [227]:
import statsmodels.api as sm

convert_old = len(df2[(df2.landing_page=='old_page')&(df2.converted == 1)])
convert_new = len(df2[(df2.landing_page=='new_page')&(df2.converted == 1)])
n_old = no # using no and nn from before
n_new = nn

m. Now use `stats.proportions_ztest` to compute your test statistic and p-value.  [Here](http://knowledgetack.com/python/statsmodels/proportions_ztest/) is a helpful link on using the built in.

In [228]:
z, p = sm.stats.proportions_ztest([convert_old,convert_new],[n_old,n_new],value=None, alternative='smaller', prop_var=False)
print(z,'\n', p)

1.3109241984234394 
 0.9050583127590245


n. What do the z-score and p-value you computed in the previous question mean for the conversion rates of the old and new pages?  Do they agree with the findings in parts **j.** and **k.**?

**Put your answer here.**
The z-score and the p value mean that we fail to reject the Null hypothesis. The p value is 0.905 and is higher than 0.05 significance level. That means we can not be confident with a 95% confidence level that the converted rate of the new_page is larger than the old_page.

<a id='regression'></a>
### Part III - A regression approach

`1.` In this final part, you will see that the result you acheived in the previous A/B test can also be acheived by performing regression.<br><br>

a. Since each row is either a conversion or no conversion, what type of regression should you be performing in this case?

**Put your answer here.**

Logistic Regression.

b. The goal is to use **statsmodels** to fit the regression model you specified in part **a.** to see if there is a significant difference in conversion based on which page a customer receives.  However, you first need to create a column for the intercept, and create a dummy variable column for which page each user received.  Add an **intercept** column, as well as an **ab_page** column, which is 1 when an individual receives the **treatment** and 0 if **control**.

In [219]:
df2['intercept'] = 1 # creating an intercept column

df2[['_','ab_page']] = pd.get_dummies(df2.group) # using pd.get_dummies method
df2.drop('_', axis=1, inplace= True)

df2.head(10)

,user_id,timestamp,group,landing_page,converted,intercept,ab_page
0,851104,2017-01-21 22:11:48.556739,control,old_page,0,1,0
1,804228,2017-01-12 08:01:45.159739,control,old_page,0,1,0
2,661590,2017-01-11 16:55:06.154213,treatment,new_page,0,1,1
3,853541,2017-01-08 18:28:03.143765,treatment,new_page,0,1,1
4,864975,2017-01-21 01:52:26.210827,control,old_page,1,1,0
5,936923,2017-01-10 15:20:49.083499,control,old_page,0,1,0
6,679687,2017-01-19 03:26:46.940749,treatment,new_page,1,1,1
7,719014,2017-01-17 01:48:29.539573,control,old_page,0,1,0
8,817355,2017-01-04 17:58:08.979471,treatment,new_page,1,1,1
9,839785,2017-01-15 18:11:06.610965,treatment,new_page,1,1,1


c. Use **statsmodels** to import your regression model.  Instantiate the model, and fit the model using the two columns you created in part **b.** to predict whether or not an individual converts.

In [220]:
lr = sm.Logit(df2.converted, df2[['intercept', 'ab_page']]).fit() # Instantiating the model, and fitting it.

preds = lr.predict(df2[['intercept', 'ab_page']]) # using the predict method to predict (obviosly)

Optimization terminated successfully.
         Current function value: 0.366118
         Iterations 6


d. Provide the summary of your model below, and use it as necessary to answer the following questions.

In [221]:
lr.summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                           Logit Regression Results                           
==============================================================================
Dep. Variable:              converted   No. Observations:               290584
Model:                          Logit   Df Residuals:                   290582
Method:                           MLE   Df Model:                            1
Date:                Mon, 19 Sep 2022   Pseudo R-squ.:               8.077e-06
Time:                        13:40:31   Log-Likelihood:            -1.0639e+05
converged:                       True   LL-Null:                   -1.0639e+05
Covariance Type:            nonrobust   LLR p-value:                    0.1899
==============================================================================
                 coef    std err          z      P>|z|      [0.025      0.975]
------------------------------------------------------------------------------
intercept     -1.9888      0.008   -246.669      0.000      -2.005      -1.973
ab_page       -0.0150      0.011     -1.311      0.190      -0.037       0.007
==============================================================================
"""

e. What is the p-value associated with **ab_page**? Why does it differ from the value you found in **Part II**?<br><br>  **Hint**: What are the null and alternative hypotheses associated with your regression model, and how do they compare to the null and alternative hypotheses in the **Part II**?

**Put your answer here.**
the p value here is 0.19, and it's different from before as the logistic regression didn't assumed the null hypothesis to be that the probability is the same for the old and new pages converting users (unlike before), and also this might be because the tests of the regression model (not the A/B test) assumes an intercept and because of differences in one or two-tailed testing.

f. Now, you are considering other things that might influence whether or not an individual converts.  Discuss why it is a good idea to consider other factors to add into your regression model.  Are there any disadvantages to adding additional terms into your regression model?

**Put your answer here.**
The user's income can has a correlation to whether a user would convert or not, I believe that the The more a user earns, the more likely they are to convert, and a disadvantage of to adding additiona terms into logistic regression is that it can reduce the power of analysis, also a common problem of adding many parameters into the regression model is multicollinearity. Multicollinearity occurs when two or more parameters are collinear with each other.

g. Now along with testing if the conversion rate changes for different pages, also add an effect based on which country a user lives. You will need to read in the **countries.csv** dataset and merge together your datasets on the approporiate rows.  [Here](https://pandas.pydata.org/pandas-docs/stable/generated/pandas.DataFrame.join.html) are the docs for joining tables. 

Does it appear that country had an impact on conversion?  Don't forget to create dummy variables for these country columns - **Hint: You will need two columns for the three dummy variables.** Provide the statistical output as well as a written response to answer this question.

In [241]:
countries_df = pd.read_csv('./countries.csv')
df_new = countries_df.set_index('user_id').join(df2.set_index('user_id'), how='inner')
df_new.head(5)

,country,timestamp,group,landing_page,converted,intercept,ab_page
user_id,,,,,,,
834778,UK,2017-01-14 23:08:43.304998,control,old_page,0,1,0
928468,US,2017-01-23 14:44:16.387854,treatment,new_page,0,1,1
822059,UK,2017-01-16 14:04:14.719771,treatment,new_page,1,1,1
711597,UK,2017-01-22 03:14:24.763511,control,old_page,0,1,0
710616,UK,2017-01-16 13:14:44.000513,treatment,new_page,0,1,1


In [242]:
### Create the necessary dummy variables
df_new = pd.concat([df_new, pd.get_dummies(df_new['country'])],axis=1).drop('country',axis=1) # using concat method to join the dummies to df directly

lr = sm.Logit(df_new.converted, df_new[['intercept', 'US', 'UK', 'ab_page']]).fit()

lr.summary()

Optimization terminated successfully.
         Current function value: 0.366113
         Iterations 6


<class 'statsmodels.iolib.summary.Summary'>
"""
                           Logit Regression Results                           
==============================================================================
Dep. Variable:              converted   No. Observations:               290584
Model:                          Logit   Df Residuals:                   290580
Method:                           MLE   Df Model:                            3
Date:                Mon, 19 Sep 2022   Pseudo R-squ.:               2.323e-05
Time:                        14:17:19   Log-Likelihood:            -1.0639e+05
converged:                       True   LL-Null:                   -1.0639e+05
Covariance Type:            nonrobust   LLR p-value:                    0.1760
==============================================================================
                 coef    std err          z      P>|z|      [0.025      0.975]
------------------------------------------------------------------------------
intercept     -2.0300      0.027    -76.249      0.000      -2.082      -1.978
US             0.0408      0.027      1.516      0.130      -0.012       0.093
UK             0.0506      0.028      1.784      0.074      -0.005       0.106
ab_page       -0.0149      0.011     -1.307      0.191      -0.037       0.007
==============================================================================
"""

The country does not appear to have influence on the convertion rate. since all the p values are above 0.05, although the CA p value is the most neer to 0.05.

h. Though you have now looked at the individual factors of country and page on conversion, we would now like to look at an interaction between page and country to see if there significant effects on conversion.  Create the necessary additional columns, and fit the new model.  

Provide the summary results, and your conclusions based on the results.

In [243]:
### Fit Your Linear Model And Obtain the Results
df_new['ab_UK'] = df_new['ab_page'] * df_new['UK']
df_new['ab_CA'] = df_new['ab_page'] * df_new['CA']

lr = logit_mod = sm.Logit(df_new['converted'], df_new[['intercept','ab_page','UK','CA','ab_UK', 'ab_CA']]).fit()

lr.summary()

Optimization terminated successfully.
         Current function value: 0.366109
         Iterations 6


<class 'statsmodels.iolib.summary.Summary'>
"""
                           Logit Regression Results                           
==============================================================================
Dep. Variable:              converted   No. Observations:               290584
Model:                          Logit   Df Residuals:                   290578
Method:                           MLE   Df Model:                            5
Date:                Mon, 19 Sep 2022   Pseudo R-squ.:               3.482e-05
Time:                        14:19:10   Log-Likelihood:            -1.0639e+05
converged:                       True   LL-Null:                   -1.0639e+05
Covariance Type:            nonrobust   LLR p-value:                    0.1920
==============================================================================
                 coef    std err          z      P>|z|      [0.025      0.975]
------------------------------------------------------------------------------
intercept     -1.9865      0.010   -206.344      0.000      -2.005      -1.968
ab_page       -0.0206      0.014     -1.505      0.132      -0.047       0.006
UK            -0.0057      0.019     -0.306      0.760      -0.043       0.031
CA            -0.0175      0.038     -0.465      0.642      -0.091       0.056
ab_UK          0.0314      0.027      1.181      0.238      -0.021       0.084
ab_CA         -0.0469      0.054     -0.872      0.383      -0.152       0.059
==============================================================================
"""

In [244]:
np.exp(lr.params).iloc[1:] # to calculate the odds ratio.

ab_page    0.979646
UK         0.994272
CA         0.982625
ab_UK      1.031896
ab_CA      0.954198
dtype: float64

#### From the summary and the odds ratio we can find that the interaction between ab_page and the countries have approximately equal odd ratio values,and the p values for both interaction terms is higher than 0.05. thus we fail to reject the null hypothesis.

<a id='conclusions'></a>
## Conclusions

I'll recommend to the company to keep theold page, as from the statiscal tests and logistic regression, I can say that both the old and new page have an approximately equal chance of converting users, so we fail to reject the null hypothesis.

Congratulations on completing the project! 

### Gather Submission Materials

Once you are satisfied with the status of your Notebook, you should save it in a format that will make it easy for others to read. You can use the __File -> Download as -> HTML (.html)__ menu to save your notebook as an .html file. If you are working locally and get an error about "No module name", then open a terminal and try installing the missing module using `pip install <module_name>` (don't include the "<" or ">" or any words following a period in the module name).

You will submit both your original Notebook and an HTML or PDF copy of the Notebook for review. There is no need for you to include any data files with your submission. If you made reference to other websites, books, and other resources to help you in solving tasks in the project, make sure that you document them. It is recommended that you either add a "Resources" section in a Markdown cell at the end of the Notebook report, or you can include a `readme.txt` file documenting your sources.

### Submit the Project

When you're ready, click on the "Submit Project" button to go to the project submission page. You can submit your files as a .zip archive or you can link to a GitHub repository containing your project files. If you go with GitHub, note that your submission will be a snapshot of the linked repository at time of submission. It is recommended that you keep each project in a separate repository to avoid any potential confusion: if a reviewer gets multiple folders representing multiple projects, there might be confusion regarding what project is to be evaluated.

It can take us up to a week to grade the project, but in most cases it is much faster. You will get an email once your submission has been reviewed. If you are having any problems submitting your project or wish to check on the status of your submission, please email us at dataanalyst-project@udacity.com. In the meantime, you should feel free to continue on with your learning journey by beginning the next module in the program.